In [1]:
import numpy as np # to handle matrix and data operation
import pandas as pd # to read csv and handle dataframe

import torch
from torch import nn, optim
import torch.nn.functional as F
import torch.utils.data
from time import time
from torchvision import datasets, transforms
from torch.autograd import Variable

from sklearn.model_selection import train_test_split

torch.set_default_tensor_type(torch.cuda.FloatTensor)


BATCH_SIZE = 32

transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

trainset = datasets.MNIST('./data/train', download=True, train=True, transform=transform)
valset = datasets.MNIST('./data/test', download=True, train=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=BATCH_SIZE, shuffle=True)


In [2]:
class Net(nn.Module):
    def __init__(self, device):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=2)
        self.pool1 = nn.MaxPool2d(2,2)
        self.pool2 = nn.MaxPool2d(2,2)
        self.linear2 = nn.Linear(6 * 6 * 16, 10)

    def forward(self,X):
        X = self.pool1(F.relu(self.conv1(X)))
        X = self.pool2(F.relu(self.conv2(X)))
        X = X.view(-1, 6 * 6 * 16)
        X = self.linear2(X)
        return X

model = Net(device='cuda:0')
print(model)

Net(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(2, 2), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear2): Linear(in_features=576, out_features=10, bias=True)
)


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
images, labels = next(iter(trainloader))
print(images.shape)
logps = model(images.to(device)) #log probabilities
loss = criterion(logps, labels.to(device)) #calculate t

torch.Size([32, 1, 28, 28])


In [4]:
def accuracy(testloader, epoch):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs= model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print("Accuracy", 100 * correct / total)
        return 100 * correct / total

# accuracy(valloader, 0)

In [5]:
optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)
time0 = time()
epochs = 2
for e in range(epochs):
    running_loss = 0
    running_val_loss = 0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        output = model(images)
        loss = criterion(output, labels)

        loss.backward()

        optimizer.step()

        running_loss += loss.item()
    for images, labels in valloader:
        images, labels = images.to(device), labels.to(device)
        output = model(images)
        loss = criterion(output, labels)
        running_val_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(e, running_loss/len(trainloader)))
        print("Epoch {} - Validation loss: {}".format(e, running_val_loss/len(valloader)))
        print("on train set:")
        accuracy(trainloader, e)
        print("on test set:")
        accuracy(valloader, e)
print("\nTraining Time (in minutes) =",(time()-time0)/60)

Epoch 0 - Training loss: 0.3448950183232625
Epoch 0 - Validation loss: 0.12025214077089541
on train set:
Accuracy 96.015
on test set:
Accuracy 96.4
Epoch 1 - Training loss: 0.10470779202381769
Epoch 1 - Validation loss: 0.07472692991788395
on train set:
Accuracy 97.57833333333333
on test set:
Accuracy 97.74

Training Time (in minutes) = 0.9309555411338806


In [6]:
PATH = './mnist_net'
torch.save(model.state_dict(), PATH)

Poniżej dane wskazujące na przetrenowania


Epoch 0 - Training loss: 0.3483577066699664
Epoch 0 - Training loss: 0.10861856871233962
on train set:
Accuracy 96.57
on test set:
Accuracy 96.76
Epoch 1 - Training loss: 0.10320847239891688
Epoch 1 - Training loss: 0.07951102484338962
on train set:
Accuracy 97.55
on test set:
Accuracy 97.52
Epoch 2 - Training loss: 0.07951653525431951
Epoch 2 - Training loss: 0.06962830385270591
on train set:
Accuracy 97.98
on test set:
Accuracy 97.79
Epoch 3 - Training loss: 0.06738914438883463
Epoch 3 - Training loss: 0.05580564471669852
on train set:
Accuracy 98.35833333333333
on test set:
Accuracy 98.2